<a href="https://colab.research.google.com/github/jinsusong/study-Kaggle/blob/main/Classification_Spooky_Author_Identification_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Spooky Author Identification 데이터 불러오기 

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
#!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json
!pwd

In [ ]:
!kaggle competitions download -c spooky-author-identification

In [ ]:
!ls -al

# 데이터 압축 해제

In [ ]:
!mkdir data
!unzip sample_submission.zip -d data/
!unzip test.zip -d data/
!unzip train.zip -d data/

# 데이터 확인하기 

In [ ]:
import pandas as pd 

label_cols = ['author']
train = pd.read_csv("data/train.csv")
print(len(train))
#len_eap = train['author'] == 'EAP'
#len_eap
train.head()



In [ ]:

test = pd.read_csv("data/test.csv")
test.head()


In [ ]:
!pip install transformers

# 버트 토크나이저 로드 

In [ ]:
from transformers import AutoTokenizer

bert_model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)


#Tokenized

In [ ]:
# train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=512,add_special_tokens=True, return_attention_mask=True) 
# val_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=512)
# len(train_encodings)
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences
import numpy as np

MAX_LEN = 128
def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
            sentence, # Sentence to encode
            add_special_tokens  = True , # Add '[CLS]' and '[SEP]'
            max_length = max_seq_len, # Truencate all sentences,
        )
        tokenized_sentences.append(tokenized_sentence)
    
    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)
        
    return np.asarray(attention_masks)

# 토크나이징 작업
input_ids = tokenize_sentences(train['text'], tokenizer, MAX_LEN)
print("토크나이저 후 " , input_ids)

# 패딩 작업 
# maxlen : 정수, 모든 시퀀스의 최대 길이
# dtype : 출력 시퀀스의 자료형. 
# padding: 'pre' 혹은 'post': 각 시퀀스의 처음 혹은 끝을 패딩
# truncating: 'pre' 혹은 'post': maxlen보다 큰 시퀀스의 처음 혹은 끝의 값들을 제거
# value: 부동소수점 혹은 문자열, 패딩할 값.

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
print("패딩 후 : ", input_ids)

# 패딩 된 데이터를 기반으로 어텐션 마스크 생성
attention_masks = create_attention_masks(input_ids)
print("어텐션 마스크 생성 :", attention_masks)

#Preprocess Data 

In [ ]:
# from sklearn.model_selection import train_test_split
# X = list(train["text"])
# y = list(train["author"])
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [ ]:
#X_train
#y_train

In [ ]:
#print(X_train_tokenized)
#print(X_train_tokenized)

In [ ]:
train['author']
labels =  train['author'].values
print(labels)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(train_inputs)
validation_size = len(validation_inputs)


#데이터 셋으로 변환

In [ ]:
# import tensorflow as tf
# train_data = tf.data.Dataset.from_tensor_slices(train_encodings)

BATCH_SIZE = 32 
NR_EPOCHS  = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    # 데이터 배열이 메모리에 존재한다면, Dataset을 만드는 가장 간단한 방법은 Dataset.from_tensor_slices()를 사용하여 tf.Tensor로 변환하는 것
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple) # from_tensor_slices : 차원을 맞춰서 데이터를 변환 ex) .slices(([1,2,],[3,4,],[5,6])) -> ((1,3,5),(2,4,6))
    print(dataset)
    return dataset 


train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

In [ ]:
# train_data

In [ ]:
# len(train_data)
# i == 0
# for element in train_data:
#   i += 1
#   print(element)
#   if i > 1 : break
outputs = model(input_ids[:1])
outputs

# Training the Model 

In [ ]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained(bert_model_name)

In [ ]:
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer='adam',loss=loss)

In [ ]:
# model.fit(train_dataset, batch_size=32, epochs=1)
# #model.fit(train_dataset, validation_data=validation_dataset, batch_size=32, epochs=1)

In [ ]:
import time 
from transformers import create_optimizer

#  steps_per_epoch : 전체데이터(train_size) // 한번 학습에 사용되는 데이터 수 (BATCH_SIZE)

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = validation_size // BATCH_SIZE

# | Loss Function : 예측 값과 실제 값의 차이를 수치로 표현, 손실함수를 최소값으로 만들기 위해 가중치를 업데이트 함 
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss') # metrics.Mean : 주어진 값의 (가중된) 평균을 계산, ex) [1, 3, 5, 7]이면 평균은 4. 가중치가 [1, 1, 0, 0]으로 지정된 경우 평균은 2
validation_loss = tf.keras.metrics.Mean(name='test_loss')
# tf.keras.metrics.Mean : 훈련 중에 메트릭이 평가될 때 에포크/단계 사이에 호출

# | Optimizer (with 1-cycle-policy) : 최적의 학습률을 찾는것 
warmup_steps = steps_per_epoch // 3 
total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps 
optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# | Metrics
# ROC-AUC Score : 결국 분류의 성능 지표로 사용되는 것, AUC(Area Under Curve) 값은 ROC 곡선 밑의 면적을 구한 것으로, 1에 가까울수록 좋은 수치
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))] # metrixs.AUC() :  리만 합계를 통해 근사 AUC (곡선 아래 영역)를 계산
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]


@tf.function
def train_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    # with ... as 구문을 사용하게 되면 파일을 열고(open) 해당 구문이 끝나면 파일이 자동으로 닫히게 됨(close). 
    # with open(파일 경로, 모드) as 파일 객체:
    # GradientTape : (자동 미분을 위해 기록)  테이프는 역방향 패스(오차 역전파)의 그래디언트를 계산하기 위해 정방향 패스에 기록할 연산을 알아야 합니다.
    # TensorFlow 는 중간 연산 과정(함수, 연산)을 테이프(tape)에 차곡차곡 기록해주는 Gradient tapes 를 제공

    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)
    # loss : 예측 값과 실제 값의 차이를 수치화함( 손실함수 )
        

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables), 1.0)
    # model.trainable_variables : 현재 모델에서의 가중치 중에서 학습해야 할 가중치는 trainable_variables에 있다.
    # tape.gradient : 모델의 훈련 가능한 변수에 대한 그래디언트를 계산
    # apply_gradients : 변수에 그라디언트를 적용


    # loss 평균 값 구함.
    train_loss(loss)

    # train_auc_metrics : AUC (곡선 아래 영역)를 계산
    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

    
@tf.function
def validation_step(model, token_ids, masks, labels):
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
    for epoch in range(epochs):
        print('=' * 50, f"EPOCH{epoch}",'='*50)
        start = time.time()


        for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
            train_step(model, token_ids, masks, labels)
            if i % 1000 == 0:
                print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
                    train_auc_metrics[i].reset_states()
        
        for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
            validation_step(model, token_ids, masks, labels)
        
        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
            validation_auc_metrics[i].reset_states()

        print('\n')

train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)









